In [123]:
import pandas as pd 
from datetime import datetime, date, timedelta
from itertools import combinations


In [274]:
DATE_FORMAT = "%Y-%m-%d"
TODAY_DT = date.today() #.strftime(DATE_FORMAT)


# 1. Loading the csv of dates

In [150]:
events_df = pd.read_csv("events.csv")
events_df.head()

,event,date,importance,category
0,Philip,1921-06-10,1,birth
1,Elizabeth,1926-04-21,1,birth
2,Engagement,1947-07-09,3,both
3,Wedding,1947-11-20,2,both
4,Charles' birth,1948-11-14,2,child


In [293]:
# adding a column dt with format datetime.date()

events_df['dt'] = events_df['date'].apply(lambda x: datetime.strptime(x, DATE_FORMAT).date())
events_df.head()

,event,date,importance,category,dt
0,Philip,1921-06-10,1,birth,1921-06-10
1,Elizabeth,1926-04-21,1,birth,1926-04-21
2,Engagement,1947-07-09,3,both,1947-07-09
3,Wedding,1947-11-20,2,both,1947-11-20
4,Charles' birth,1948-11-14,2,child,1948-11-14


In [295]:
events = events_df.set_index('event').transpose().to_dict()
events

{'Philip': {'date': '1921-06-10',
  'importance': 1,
  'category': 'birth',
  'dt': datetime.date(1921, 6, 10)},
 'Elizabeth': {'date': '1926-04-21',
  'importance': 1,
  'category': 'birth',
  'dt': datetime.date(1926, 4, 21)},
 'Engagement': {'date': '1947-07-09',
  'importance': 3,
  'category': 'both',
  'dt': datetime.date(1947, 7, 9)},
 'Wedding': {'date': '1947-11-20',
  'importance': 2,
  'category': 'both',
  'dt': datetime.date(1947, 11, 20)},
 "Charles' birth": {'date': '1948-11-14',
  'importance': 2,
  'category': 'child',
  'dt': datetime.date(1948, 11, 14)},
 "Anne's birth": {'date': '1950-08-15',
  'importance': 2,
  'category': 'child',
  'dt': datetime.date(1950, 8, 15)},
 'Beginning of reign': {'date': '1952-02-06',
  'importance': 1,
  'category': 'both',
  'dt': datetime.date(1952, 2, 6)},
 'Coronation': {'date': '1953-06-02',
  'importance': 2,
  'category': 'both',
  'dt': datetime.date(1953, 6, 2)},
 "Andrew's birth": {'date': '1960-02-19',
  'importance': 2,
  

# 2. Functions

In [ ]:
def stod(dt_str):
    """Converts string variable `dt_str` to date.
    
    Args:
        dt_str (str): The string date in `DATE_FORMAT` format.
        
    Returns:
        Date
    """
    return datetime.strptime(dt_str, DATE_FORMAT).date()

In [273]:
def days_between(dt1, dt2):
    """Calculates difference between `dt1` and `dt2`.
    
    Args:
        dt1 (date): The first date.
        dt2 (date): The second date.
        
    Returns:
        Boolean
    """
    try:
        return abs((dt2 - dt1).days)
    except:
        return None


In [271]:
# iterator by date
# input and output format: date(YYYY, MM, DD) 

def daterange(start_dt, end_dt):
    """Generates a range of dates from `start_dt` to `end_dt` (including the ends).
    
    Args:
        start_dt (date): The start date.
        end_dt (date): The end date.
        
    Returns:
        Iterator of dates (date)
    """
    for n in range(int((end_dt - start_dt).days) + 1):
        yield start_dt + timedelta(n)


In [236]:
# rules for detecting anniversaries
# input format: date(YYYY, MM, DD) 

def rule_multiple(dt1, dt2, n):
    """Identifies if difference between 2 dates `dt1` and `dt2` is multiple of `n`.
    
    Args:
        dt1 (date): 
        dt2 (date): 
        n (int): The number of multiplicity.
        
    Returns:
        Boolean
    """
    if days_between(dt1, dt2) % n == 0:
        return True
    else:
        return False

def rule_anniversary(dt1, dt2):
    """Identifies if `dt1` is an anniversary of `dt2` or vice versa.
    
    Args:
        dt1 (date): 
        dt2 (date): 
        
    Returns:
        Int or None
    """
    if dt1.month == dt2.month and dt1.day == dt2.day:
        return abs(dt1.year - dt2.year)
    else:
        return None


In [133]:
def today_counter():
    print("Today is...")
    for event in events.keys():
        print("{} days since {}".format(days_between(TODAY_DT, events[event]['dt']), event))

def range_calendar(dt_start, dt_end):
    for dt in daterange(dt_start, dt_end):
        for event in events.keys():
            event_dt = events[event]['dt']
            if (
                (rule_multiple(dt, event_dt, 100) and events[event]['importance'] <= 2)
                or
                (rule_multiple(dt, event_dt, 1000) and events[event]['importance'] == 3)
            ):
                print("{} -- {} days since {}".format(dt, days_between(dt, event_dt), event))
            if rule_anniversary(dt, event_dt):
                print("{} -- {} years since {}".format(dt, rule_anniversary(dt, event_dt), event))
            


In [275]:
def differences_inside_set():
    for event1 in events.keys():
        for event2 in events.keys():
            if (
                days_between(events[event1]['dt'], events[event2]['dt']) % 100 == 0 and 
                event1 != event2 and 
                events[event1]['dt'] < events[event2]['dt']
            ):
                print("{} days between {} and {}".format(days_between(events[event1]['dt'], events[event2]['dt']), event1, event2))


In [228]:
def birth_dates():
    birth_dict = {}
    for k, v in events.items():
        if v['category'] == 'birth':
            birth_dict[k] = v['dt']    
    return birth_dict

In [208]:
def age_counter(given_dt):
    
    birth_dict = birth_dates()
    output_dict = {}
    
    try:
        for k in birth_dict.keys():
#             print("{0}'s age: {1} days".format(k, days_between(given_dt, birth_dict[k])))
            output_dict["{0}'s age".format(k)] = days_between(given_dt, birth_dict[k])

        if len(birth_dict.keys()) > 1:
            for (k1, k2) in combinations(birth_dict.keys(), 2):
#                 print("Age difference between {0} and {1}: {2} days".format(k1, k2, days_between(birth_dict[k1], birth_dict[k2])))
                output_dict["Age difference between {0} and {1}".format(k1, k2)] = days_between(birth_dict[k1], birth_dict[k2])
#             print("Total age: {} days".format(sum([days_between(given_dt, birth_dict[k]) for k in birth_dict.keys()])))
            output_dict["Total age"] = sum([days_between(given_dt, birth_dict[k]) for k in birth_dict.keys()])
        print("Ages were calculated.")
    
    except:
        print("Something wrong happened...")

    if len(output_dict):
        return pd.DataFrame.from_dict(output_dict, orient='index', columns=['days'])
    else:
        return "Sorry, there is no answer :("
    

In [233]:
def total_age_anniversaries(start_dt, end_dt, n=1000):
    """Returns dates between `start_dt` and `end_dt` in which total age (in days) 
    is close to be multiple of `n`.
    
    Args:
        start_dt (date): 
        end_dt (date): 
        n (int): The number for checking multiplicity (more or equal 10). Default value is 1000.
    
    Returns:
        DataFrame
    """
    birth_dict = birth_dates()
    output_dict = {}
    
    try:
        for dt in daterange(start_dt, end_dt):
            total_age = sum([days_between(dt, birth_dict[k]) for k in birth_dict.keys()])
            if n >= 10 and total_age % n in range(len(birth_dict)):
#                 print("{} -- {} days of their combined age".format(dt.strftime('%Y-%m-%d'), total_age))
                output_dict[dt.strftime('%Y-%m-%d')] = total_age
    except:
        print("Something wrong happened...")
    
    if len(output_dict):
        return pd.DataFrame.from_dict(output_dict, orient='index', columns=['days'])
    else:
        return "Sorry, there is no answer :("
    

# 3. Performing

In [211]:
# given_dt = stod(events["Philip's death"]['date'])

# age_counter(given_dt)
# age_counter("")

# ok

In [235]:
# start_dt = stod(events['Engagement']['date'])
# end_dt = stod(events["Philip's death"]['date'])

# total_age_anniversaries(start_dt, end_dt)
# total_age_anniversaries(start_dt, "")

# ok

In [161]:
# today_counter(events)

In [160]:
# range_calendar(date.today(), date(2023, 12, 31))


In [129]:
# birthday_dt_1 = events["Elizabeth"]['date']
# birthday_dt_2 = events["Philip"]['date']
# given_dt = events["Philip's death"]['date']


In [157]:
# total_age_anniversaries(birthday_dt_1, birthday_dt_2, events['Engagement']['date'], events["Philip's death"]['date'])


In [156]:
# differences_inside_set(events)

In [ ]:
# str -> date: datetime.strptime(dt, "%Y-%m-%d").date()
# date -> str: dt.strftime('%Y-%m-%d')

#  in 'YYYY-MM-DD' format

In [212]:
if __name__=="__main__":
    print(age_counter('2021-04-09'))

Ages were calculated.
                                              days
Philip's age                                 36463
Elizabeth's age                              34687
Age difference between Philip and Elizabeth   1776
Total age                                    71150
